In [1]:
import pandas as pd
import warnings
import re
import numpy as np
import lightgbm as lgb
from IPython.display import HTML
from sklearn.metrics import roc_auc_score
# 描画関連
pd.options.display.precision = 15
warnings.filterwarnings("ignore")
# COL数
pd.set_option("display.max_columns", None)
# ROW数
pd.set_option('display.max_rows', 1000)
# 幅
display(HTML("<style>.container { width:100% !important; }</style>"))

# 最後の出力
print('All components loads was done! Ready!')

All components loads was done! Ready!


In [2]:
df = pd.read_csv("./var/collect_data.csv")

In [3]:
# uniq_key -> hd, jcd, rno

def split_uniq_key(df, index=0):
    kv = df["uniq_key"].apply(lambda x:x.split("&")[index])
    value = kv.apply(lambda x:x.split("=")[-1])
    return value

df["rno"] = split_uniq_key(df, index=0)
df["jcd"] = split_uniq_key(df, index=1)
df["hd"] = split_uniq_key(df, index=2)

df.sort_values(by=["hd", "rno"], inplace=True)
df

,uniq_key,rank,race_time,tansho_odds,fukusho_odds,cup_name,kaijo_name,waku_name,touroku_no,class_name,racer_name,country,age,weight,fnum,lnum,mean_st,zenkoku_win_in_1_prob,zenkoku_win_in_2_prob,zenkoku_win_in_3_prob,touchi_win_in_1_prob,touchi_win_in_2_prob,touchi_win_in_3_prob,mortor_win_in_1_prob,mortor_win_in_2_prob,mortor_win_in_3_prob,boat_win_in_1_prob,boat_win_in_2_prob,boat_win_in_3_prob,rno,jcd,hd
951840,rno=12&jcd=05&hd=20110507,3,"1'49""8",NaN,NaN,東日本大震災被災地支援競走第４６回東京スポーツ賞,多摩川,1,3484,A1,芝田 浩治,兵庫,兵庫39歳,53.5kg,F0,L0,0.18,7.22,56.200000000000003,0.000000000000000,6.64,57.500000000000000,0.000000000000000,22,22.600000000000001,-,66,29.600000000000001,-,12,05,20110507
951841,rno=12&jcd=05&hd=20110507,5,"1'51""4",NaN,NaN,東日本大震災被災地支援競走第４６回東京スポーツ賞,多摩川,2,3933,A1,山地 正樹,岡山,岡山34歳,53.5kg,F0,L0,0.18,7.11,56.399999999999999,0.000000000000000,6.89,52.600000000000001,0.000000000000000,36,31.600000000000001,-,57,33.500000000000000,-,12,05,20110507
951842,rno=12&jcd=05&hd=20110507,2,"1'48""6",NaN,NaN,東日本大震災被災地支援競走第４６回東京スポーツ賞,多摩川,3,3563,A1,堤 昇,静岡,静岡41歳,50.1kg,F0,L0,0.16,6.61,49.500000000000000,0.000000000000000,6.29,51.600000000000001,0.000000000000000,28,35.700000000000003,-,23,30.600000000000001,-,12,05,20110507
951843,rno=12&jcd=05&hd=20110507,1,"1'47""0",NaN,NaN,東日本大震災被災地支援競走第４６回東京スポーツ賞,多摩川,4,3986,A1,沖島 広和,福岡,福岡33歳,52.9kg,F0,L0,0.16,7.03,52.799999999999997,0.000000000000000,6.70,60.000000000000000,0.000000000000000,55,27.199999999999999,-,28,41.299999999999997,-,12,05,20110507
951844,rno=12&jcd=05&hd=20110507,6,"1'52""6",NaN,NaN,東日本大震災被災地支援競走第４６回東京スポーツ賞,多摩川,5,4089,A1,妹尾 忠幸,岡山,岡山30歳,53.5kg,F0,L0,0.18,6.15,43.899999999999999,0.000000000000000,5.90,41.299999999999997,0.000000000000000,25,38.200000000000003,-,60,26.899999999999999,-,12,05,20110507
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996079,rno=9&jcd=13&hd=20200715,1,"1'50""9",1.3,1.2-1.4,尼崎モーターボート大賞〜まくってちょ〜うだい！！〜,尼崎,2,4477,A1,篠崎 仁志,福岡,福岡32歳,51.3kg,F0,L0,0.15,7.40,52.710000000000001,79.069999999999993,7.60,55.320000000000000,72.340000000000003,14,55.100000000000001,67.35,32,36.000000000000000,56.00,9,13,20200715
996080,rno=9&jcd=13&hd=20200715,2,"1'51""6",7.6,1.5-2.0,尼崎モーターボート大賞〜まくってちょ〜うだい！！〜,尼崎,3,4573,A1,佐藤 翼,埼玉,埼玉31歳,51.2kg,F1,L0,0.15,7.14,55.100000000000001,71.430000000000007,7.70,65.000000000000000,70.000000000000000,42,48.780000000000001,65.85,55,34.880000000000003,55.81,9,13,20200715
996081,rno=9&jcd=13&hd=20200715,5,"1'54""7",8.5,2.4-3.5,尼崎モーターボート大賞〜まくってちょ〜うだい！！〜,尼崎,4,4375,A2,照屋 厚仁,兵庫,大阪36歳,51.0kg,F0,L0,0.18,5.60,36.789999999999999,59.430000000000000,6.60,55.060000000000002,67.420000000000002,32,37.500000000000000,47.50,69,48.939999999999998,59.57,9,13,20200715
996082,rno=9&jcd=13&hd=20200715,3,"1'53""3",15.2,3.2-4.6,尼崎モーターボート大賞〜まくってちょ〜うだい！！〜,尼崎,5,4381,A2,田路 朋史,兵庫,兵庫35歳,51.0kg,F0,L0,0.17,5.65,42.049999999999997,59.090000000000003,5.73,39.799999999999997,60.200000000000003,45,33.329999999999998,51.67,62,32.649999999999999,46.94,9,13,20200715


In [4]:
# race_time sanitize

def race_time_sanitize(x):
    if not pd.isna(x):
        h, t = x.split("'")
        t = t.replace('"', ".")
        return int(h) * 60 + float(t)
    else:
        return None
df["race_time"] = df["race_time"].apply(race_time_sanitize)

df

,uniq_key,rank,race_time,tansho_odds,fukusho_odds,cup_name,kaijo_name,waku_name,touroku_no,class_name,racer_name,country,age,weight,fnum,lnum,mean_st,zenkoku_win_in_1_prob,zenkoku_win_in_2_prob,zenkoku_win_in_3_prob,touchi_win_in_1_prob,touchi_win_in_2_prob,touchi_win_in_3_prob,mortor_win_in_1_prob,mortor_win_in_2_prob,mortor_win_in_3_prob,boat_win_in_1_prob,boat_win_in_2_prob,boat_win_in_3_prob,rno,jcd,hd
951840,rno=12&jcd=05&hd=20110507,3,109.799999999999997,NaN,NaN,東日本大震災被災地支援競走第４６回東京スポーツ賞,多摩川,1,3484,A1,芝田 浩治,兵庫,兵庫39歳,53.5kg,F0,L0,0.18,7.22,56.200000000000003,0.000000000000000,6.64,57.500000000000000,0.000000000000000,22,22.600000000000001,-,66,29.600000000000001,-,12,05,20110507
951841,rno=12&jcd=05&hd=20110507,5,111.400000000000006,NaN,NaN,東日本大震災被災地支援競走第４６回東京スポーツ賞,多摩川,2,3933,A1,山地 正樹,岡山,岡山34歳,53.5kg,F0,L0,0.18,7.11,56.399999999999999,0.000000000000000,6.89,52.600000000000001,0.000000000000000,36,31.600000000000001,-,57,33.500000000000000,-,12,05,20110507
951842,rno=12&jcd=05&hd=20110507,2,108.599999999999994,NaN,NaN,東日本大震災被災地支援競走第４６回東京スポーツ賞,多摩川,3,3563,A1,堤 昇,静岡,静岡41歳,50.1kg,F0,L0,0.16,6.61,49.500000000000000,0.000000000000000,6.29,51.600000000000001,0.000000000000000,28,35.700000000000003,-,23,30.600000000000001,-,12,05,20110507
951843,rno=12&jcd=05&hd=20110507,1,107.000000000000000,NaN,NaN,東日本大震災被災地支援競走第４６回東京スポーツ賞,多摩川,4,3986,A1,沖島 広和,福岡,福岡33歳,52.9kg,F0,L0,0.16,7.03,52.799999999999997,0.000000000000000,6.70,60.000000000000000,0.000000000000000,55,27.199999999999999,-,28,41.299999999999997,-,12,05,20110507
951844,rno=12&jcd=05&hd=20110507,6,112.599999999999994,NaN,NaN,東日本大震災被災地支援競走第４６回東京スポーツ賞,多摩川,5,4089,A1,妹尾 忠幸,岡山,岡山30歳,53.5kg,F0,L0,0.18,6.15,43.899999999999999,0.000000000000000,5.90,41.299999999999997,0.000000000000000,25,38.200000000000003,-,60,26.899999999999999,-,12,05,20110507
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996079,rno=9&jcd=13&hd=20200715,1,110.900000000000006,1.3,1.2-1.4,尼崎モーターボート大賞〜まくってちょ〜うだい！！〜,尼崎,2,4477,A1,篠崎 仁志,福岡,福岡32歳,51.3kg,F0,L0,0.15,7.40,52.710000000000001,79.069999999999993,7.60,55.320000000000000,72.340000000000003,14,55.100000000000001,67.35,32,36.000000000000000,56.00,9,13,20200715
996080,rno=9&jcd=13&hd=20200715,2,111.599999999999994,7.6,1.5-2.0,尼崎モーターボート大賞〜まくってちょ〜うだい！！〜,尼崎,3,4573,A1,佐藤 翼,埼玉,埼玉31歳,51.2kg,F1,L0,0.15,7.14,55.100000000000001,71.430000000000007,7.70,65.000000000000000,70.000000000000000,42,48.780000000000001,65.85,55,34.880000000000003,55.81,9,13,20200715
996081,rno=9&jcd=13&hd=20200715,5,114.700000000000003,8.5,2.4-3.5,尼崎モーターボート大賞〜まくってちょ〜うだい！！〜,尼崎,4,4375,A2,照屋 厚仁,兵庫,大阪36歳,51.0kg,F0,L0,0.18,5.60,36.789999999999999,59.430000000000000,6.60,55.060000000000002,67.420000000000002,32,37.500000000000000,47.50,69,48.939999999999998,59.57,9,13,20200715
996082,rno=9&jcd=13&hd=20200715,3,113.299999999999997,15.2,3.2-4.6,尼崎モーターボート大賞〜まくってちょ〜うだい！！〜,尼崎,5,4381,A2,田路 朋史,兵庫,兵庫35歳,51.0kg,F0,L0,0.17,5.65,42.049999999999997,59.090000000000003,5.73,39.799999999999997,60.200000000000003,45,33.329999999999998,51.67,62,32.649999999999999,46.94,9,13,20200715


In [5]:
# split fukusho to min, max
def split_fukusho_to_min_max(x, index=0):
    if not pd.isna(x):
        try:
            min_, max_ = x.split("-")
            min_, max_ = [float(min_), float(max_)]
            return [min_, max_][index]
        except:
            return None
    else:
        return None

df["fukusho_min"] = df["fukusho_odds"].apply(lambda x:split_fukusho_to_min_max(x, 0))
df["fukusho_max"] = df["fukusho_odds"].apply(lambda x:split_fukusho_to_min_max(x, 1))

df.drop(["fukusho_odds"], axis=1, inplace=True)
df

,uniq_key,rank,race_time,tansho_odds,cup_name,kaijo_name,waku_name,touroku_no,class_name,racer_name,country,age,weight,fnum,lnum,mean_st,zenkoku_win_in_1_prob,zenkoku_win_in_2_prob,zenkoku_win_in_3_prob,touchi_win_in_1_prob,touchi_win_in_2_prob,touchi_win_in_3_prob,mortor_win_in_1_prob,mortor_win_in_2_prob,mortor_win_in_3_prob,boat_win_in_1_prob,boat_win_in_2_prob,boat_win_in_3_prob,rno,jcd,hd,fukusho_min,fukusho_max
951840,rno=12&jcd=05&hd=20110507,3,109.799999999999997,NaN,東日本大震災被災地支援競走第４６回東京スポーツ賞,多摩川,1,3484,A1,芝田 浩治,兵庫,兵庫39歳,53.5kg,F0,L0,0.18,7.22,56.200000000000003,0.000000000000000,6.64,57.500000000000000,0.000000000000000,22,22.600000000000001,-,66,29.600000000000001,-,12,05,20110507,NaN,NaN
951841,rno=12&jcd=05&hd=20110507,5,111.400000000000006,NaN,東日本大震災被災地支援競走第４６回東京スポーツ賞,多摩川,2,3933,A1,山地 正樹,岡山,岡山34歳,53.5kg,F0,L0,0.18,7.11,56.399999999999999,0.000000000000000,6.89,52.600000000000001,0.000000000000000,36,31.600000000000001,-,57,33.500000000000000,-,12,05,20110507,NaN,NaN
951842,rno=12&jcd=05&hd=20110507,2,108.599999999999994,NaN,東日本大震災被災地支援競走第４６回東京スポーツ賞,多摩川,3,3563,A1,堤 昇,静岡,静岡41歳,50.1kg,F0,L0,0.16,6.61,49.500000000000000,0.000000000000000,6.29,51.600000000000001,0.000000000000000,28,35.700000000000003,-,23,30.600000000000001,-,12,05,20110507,NaN,NaN
951843,rno=12&jcd=05&hd=20110507,1,107.000000000000000,NaN,東日本大震災被災地支援競走第４６回東京スポーツ賞,多摩川,4,3986,A1,沖島 広和,福岡,福岡33歳,52.9kg,F0,L0,0.16,7.03,52.799999999999997,0.000000000000000,6.70,60.000000000000000,0.000000000000000,55,27.199999999999999,-,28,41.299999999999997,-,12,05,20110507,NaN,NaN
951844,rno=12&jcd=05&hd=20110507,6,112.599999999999994,NaN,東日本大震災被災地支援競走第４６回東京スポーツ賞,多摩川,5,4089,A1,妹尾 忠幸,岡山,岡山30歳,53.5kg,F0,L0,0.18,6.15,43.899999999999999,0.000000000000000,5.90,41.299999999999997,0.000000000000000,25,38.200000000000003,-,60,26.899999999999999,-,12,05,20110507,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996079,rno=9&jcd=13&hd=20200715,1,110.900000000000006,1.3,尼崎モーターボート大賞〜まくってちょ〜うだい！！〜,尼崎,2,4477,A1,篠崎 仁志,福岡,福岡32歳,51.3kg,F0,L0,0.15,7.40,52.710000000000001,79.069999999999993,7.60,55.320000000000000,72.340000000000003,14,55.100000000000001,67.35,32,36.000000000000000,56.00,9,13,20200715,1.2,1.4
996080,rno=9&jcd=13&hd=20200715,2,111.599999999999994,7.6,尼崎モーターボート大賞〜まくってちょ〜うだい！！〜,尼崎,3,4573,A1,佐藤 翼,埼玉,埼玉31歳,51.2kg,F1,L0,0.15,7.14,55.100000000000001,71.430000000000007,7.70,65.000000000000000,70.000000000000000,42,48.780000000000001,65.85,55,34.880000000000003,55.81,9,13,20200715,1.5,2.0
996081,rno=9&jcd=13&hd=20200715,5,114.700000000000003,8.5,尼崎モーターボート大賞〜まくってちょ〜うだい！！〜,尼崎,4,4375,A2,照屋 厚仁,兵庫,大阪36歳,51.0kg,F0,L0,0.18,5.60,36.789999999999999,59.430000000000000,6.60,55.060000000000002,67.420000000000002,32,37.500000000000000,47.50,69,48.939999999999998,59.57,9,13,20200715,2.4,3.5
996082,rno=9&jcd=13&hd=20200715,3,113.299999999999997,15.2,尼崎モーターボート大賞〜まくってちょ〜うだい！！〜,尼崎,5,4381,A2,田路 朋史,兵庫,兵庫35歳,51.0kg,F0,L0,0.17,5.65,42.049999999999997,59.090000000000003,5.73,39.799999999999997,60.200000000000003,45,33.329999999999998,51.67,62,32.649999999999999,46.94,9,13,20200715,3.2,4.6


In [6]:
# age sanitize
def age_sanitize(x):
    try:
        return int(re.search("(\d{1,})", x).group(1))
    except:
        return None

df["age"] = df["age"].apply(age_sanitize)
df

,uniq_key,rank,race_time,tansho_odds,cup_name,kaijo_name,waku_name,touroku_no,class_name,racer_name,country,age,weight,fnum,lnum,mean_st,zenkoku_win_in_1_prob,zenkoku_win_in_2_prob,zenkoku_win_in_3_prob,touchi_win_in_1_prob,touchi_win_in_2_prob,touchi_win_in_3_prob,mortor_win_in_1_prob,mortor_win_in_2_prob,mortor_win_in_3_prob,boat_win_in_1_prob,boat_win_in_2_prob,boat_win_in_3_prob,rno,jcd,hd,fukusho_min,fukusho_max
951840,rno=12&jcd=05&hd=20110507,3,109.799999999999997,NaN,東日本大震災被災地支援競走第４６回東京スポーツ賞,多摩川,1,3484,A1,芝田 浩治,兵庫,39,53.5kg,F0,L0,0.18,7.22,56.200000000000003,0.000000000000000,6.64,57.500000000000000,0.000000000000000,22,22.600000000000001,-,66,29.600000000000001,-,12,05,20110507,NaN,NaN
951841,rno=12&jcd=05&hd=20110507,5,111.400000000000006,NaN,東日本大震災被災地支援競走第４６回東京スポーツ賞,多摩川,2,3933,A1,山地 正樹,岡山,34,53.5kg,F0,L0,0.18,7.11,56.399999999999999,0.000000000000000,6.89,52.600000000000001,0.000000000000000,36,31.600000000000001,-,57,33.500000000000000,-,12,05,20110507,NaN,NaN
951842,rno=12&jcd=05&hd=20110507,2,108.599999999999994,NaN,東日本大震災被災地支援競走第４６回東京スポーツ賞,多摩川,3,3563,A1,堤 昇,静岡,41,50.1kg,F0,L0,0.16,6.61,49.500000000000000,0.000000000000000,6.29,51.600000000000001,0.000000000000000,28,35.700000000000003,-,23,30.600000000000001,-,12,05,20110507,NaN,NaN
951843,rno=12&jcd=05&hd=20110507,1,107.000000000000000,NaN,東日本大震災被災地支援競走第４６回東京スポーツ賞,多摩川,4,3986,A1,沖島 広和,福岡,33,52.9kg,F0,L0,0.16,7.03,52.799999999999997,0.000000000000000,6.70,60.000000000000000,0.000000000000000,55,27.199999999999999,-,28,41.299999999999997,-,12,05,20110507,NaN,NaN
951844,rno=12&jcd=05&hd=20110507,6,112.599999999999994,NaN,東日本大震災被災地支援競走第４６回東京スポーツ賞,多摩川,5,4089,A1,妹尾 忠幸,岡山,30,53.5kg,F0,L0,0.18,6.15,43.899999999999999,0.000000000000000,5.90,41.299999999999997,0.000000000000000,25,38.200000000000003,-,60,26.899999999999999,-,12,05,20110507,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996079,rno=9&jcd=13&hd=20200715,1,110.900000000000006,1.3,尼崎モーターボート大賞〜まくってちょ〜うだい！！〜,尼崎,2,4477,A1,篠崎 仁志,福岡,32,51.3kg,F0,L0,0.15,7.40,52.710000000000001,79.069999999999993,7.60,55.320000000000000,72.340000000000003,14,55.100000000000001,67.35,32,36.000000000000000,56.00,9,13,20200715,1.2,1.4
996080,rno=9&jcd=13&hd=20200715,2,111.599999999999994,7.6,尼崎モーターボート大賞〜まくってちょ〜うだい！！〜,尼崎,3,4573,A1,佐藤 翼,埼玉,31,51.2kg,F1,L0,0.15,7.14,55.100000000000001,71.430000000000007,7.70,65.000000000000000,70.000000000000000,42,48.780000000000001,65.85,55,34.880000000000003,55.81,9,13,20200715,1.5,2.0
996081,rno=9&jcd=13&hd=20200715,5,114.700000000000003,8.5,尼崎モーターボート大賞〜まくってちょ〜うだい！！〜,尼崎,4,4375,A2,照屋 厚仁,兵庫,36,51.0kg,F0,L0,0.18,5.60,36.789999999999999,59.430000000000000,6.60,55.060000000000002,67.420000000000002,32,37.500000000000000,47.50,69,48.939999999999998,59.57,9,13,20200715,2.4,3.5
996082,rno=9&jcd=13&hd=20200715,3,113.299999999999997,15.2,尼崎モーターボート大賞〜まくってちょ〜うだい！！〜,尼崎,5,4381,A2,田路 朋史,兵庫,35,51.0kg,F0,L0,0.17,5.65,42.049999999999997,59.090000000000003,5.73,39.799999999999997,60.200000000000003,45,33.329999999999998,51.67,62,32.649999999999999,46.94,9,13,20200715,3.2,4.6


In [7]:
# weight sanitize
def weight_sanitize(x):
    try:
        return re.search("(\d\d.\d)", x).group(1)
    except:
        return None
df["weight"] = df["weight"].apply(weight_sanitize)
df

,uniq_key,rank,race_time,tansho_odds,cup_name,kaijo_name,waku_name,touroku_no,class_name,racer_name,country,age,weight,fnum,lnum,mean_st,zenkoku_win_in_1_prob,zenkoku_win_in_2_prob,zenkoku_win_in_3_prob,touchi_win_in_1_prob,touchi_win_in_2_prob,touchi_win_in_3_prob,mortor_win_in_1_prob,mortor_win_in_2_prob,mortor_win_in_3_prob,boat_win_in_1_prob,boat_win_in_2_prob,boat_win_in_3_prob,rno,jcd,hd,fukusho_min,fukusho_max
951840,rno=12&jcd=05&hd=20110507,3,109.799999999999997,NaN,東日本大震災被災地支援競走第４６回東京スポーツ賞,多摩川,1,3484,A1,芝田 浩治,兵庫,39,53.5,F0,L0,0.18,7.22,56.200000000000003,0.000000000000000,6.64,57.500000000000000,0.000000000000000,22,22.600000000000001,-,66,29.600000000000001,-,12,05,20110507,NaN,NaN
951841,rno=12&jcd=05&hd=20110507,5,111.400000000000006,NaN,東日本大震災被災地支援競走第４６回東京スポーツ賞,多摩川,2,3933,A1,山地 正樹,岡山,34,53.5,F0,L0,0.18,7.11,56.399999999999999,0.000000000000000,6.89,52.600000000000001,0.000000000000000,36,31.600000000000001,-,57,33.500000000000000,-,12,05,20110507,NaN,NaN
951842,rno=12&jcd=05&hd=20110507,2,108.599999999999994,NaN,東日本大震災被災地支援競走第４６回東京スポーツ賞,多摩川,3,3563,A1,堤 昇,静岡,41,50.1,F0,L0,0.16,6.61,49.500000000000000,0.000000000000000,6.29,51.600000000000001,0.000000000000000,28,35.700000000000003,-,23,30.600000000000001,-,12,05,20110507,NaN,NaN
951843,rno=12&jcd=05&hd=20110507,1,107.000000000000000,NaN,東日本大震災被災地支援競走第４６回東京スポーツ賞,多摩川,4,3986,A1,沖島 広和,福岡,33,52.9,F0,L0,0.16,7.03,52.799999999999997,0.000000000000000,6.70,60.000000000000000,0.000000000000000,55,27.199999999999999,-,28,41.299999999999997,-,12,05,20110507,NaN,NaN
951844,rno=12&jcd=05&hd=20110507,6,112.599999999999994,NaN,東日本大震災被災地支援競走第４６回東京スポーツ賞,多摩川,5,4089,A1,妹尾 忠幸,岡山,30,53.5,F0,L0,0.18,6.15,43.899999999999999,0.000000000000000,5.90,41.299999999999997,0.000000000000000,25,38.200000000000003,-,60,26.899999999999999,-,12,05,20110507,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996079,rno=9&jcd=13&hd=20200715,1,110.900000000000006,1.3,尼崎モーターボート大賞〜まくってちょ〜うだい！！〜,尼崎,2,4477,A1,篠崎 仁志,福岡,32,51.3,F0,L0,0.15,7.40,52.710000000000001,79.069999999999993,7.60,55.320000000000000,72.340000000000003,14,55.100000000000001,67.35,32,36.000000000000000,56.00,9,13,20200715,1.2,1.4
996080,rno=9&jcd=13&hd=20200715,2,111.599999999999994,7.6,尼崎モーターボート大賞〜まくってちょ〜うだい！！〜,尼崎,3,4573,A1,佐藤 翼,埼玉,31,51.2,F1,L0,0.15,7.14,55.100000000000001,71.430000000000007,7.70,65.000000000000000,70.000000000000000,42,48.780000000000001,65.85,55,34.880000000000003,55.81,9,13,20200715,1.5,2.0
996081,rno=9&jcd=13&hd=20200715,5,114.700000000000003,8.5,尼崎モーターボート大賞〜まくってちょ〜うだい！！〜,尼崎,4,4375,A2,照屋 厚仁,兵庫,36,51.0,F0,L0,0.18,5.60,36.789999999999999,59.430000000000000,6.60,55.060000000000002,67.420000000000002,32,37.500000000000000,47.50,69,48.939999999999998,59.57,9,13,20200715,2.4,3.5
996082,rno=9&jcd=13&hd=20200715,3,113.299999999999997,15.2,尼崎モーターボート大賞〜まくってちょ〜うだい！！〜,尼崎,5,4381,A2,田路 朋史,兵庫,35,51.0,F0,L0,0.17,5.65,42.049999999999997,59.090000000000003,5.73,39.799999999999997,60.200000000000003,45,33.329999999999998,51.67,62,32.649999999999999,46.94,9,13,20200715,3.2,4.6


In [8]:
df.to_csv("boatrace.csv.gz", compression="gzip", index=None)

In [9]:
df["kaijo_name"].dtype.str

'|O'

In [10]:
# label encoding
for cat_col in ["class_name", "kaijo_name", "racer_name", "country", "fnum", "lnum"]:
    if df[cat_col].dtype.str == "|O":
        dic = {val:idx for idx, val in enumerate(df[cat_col].unique())}
        df[cat_col] = df[cat_col].apply(lambda x:dic[x])

df

,uniq_key,rank,race_time,tansho_odds,cup_name,kaijo_name,waku_name,touroku_no,class_name,racer_name,country,age,weight,fnum,lnum,mean_st,zenkoku_win_in_1_prob,zenkoku_win_in_2_prob,zenkoku_win_in_3_prob,touchi_win_in_1_prob,touchi_win_in_2_prob,touchi_win_in_3_prob,mortor_win_in_1_prob,mortor_win_in_2_prob,mortor_win_in_3_prob,boat_win_in_1_prob,boat_win_in_2_prob,boat_win_in_3_prob,rno,jcd,hd,fukusho_min,fukusho_max
951840,rno=12&jcd=05&hd=20110507,3,109.799999999999997,NaN,東日本大震災被災地支援競走第４６回東京スポーツ賞,0,1,3484,0,0,0,39,53.5,0,0,0.18,7.22,56.200000000000003,0.000000000000000,6.64,57.500000000000000,0.000000000000000,22,22.600000000000001,-,66,29.600000000000001,-,12,05,20110507,NaN,NaN
951841,rno=12&jcd=05&hd=20110507,5,111.400000000000006,NaN,東日本大震災被災地支援競走第４６回東京スポーツ賞,0,2,3933,0,1,1,34,53.5,0,0,0.18,7.11,56.399999999999999,0.000000000000000,6.89,52.600000000000001,0.000000000000000,36,31.600000000000001,-,57,33.500000000000000,-,12,05,20110507,NaN,NaN
951842,rno=12&jcd=05&hd=20110507,2,108.599999999999994,NaN,東日本大震災被災地支援競走第４６回東京スポーツ賞,0,3,3563,0,2,2,41,50.1,0,0,0.16,6.61,49.500000000000000,0.000000000000000,6.29,51.600000000000001,0.000000000000000,28,35.700000000000003,-,23,30.600000000000001,-,12,05,20110507,NaN,NaN
951843,rno=12&jcd=05&hd=20110507,1,107.000000000000000,NaN,東日本大震災被災地支援競走第４６回東京スポーツ賞,0,4,3986,0,3,3,33,52.9,0,0,0.16,7.03,52.799999999999997,0.000000000000000,6.70,60.000000000000000,0.000000000000000,55,27.199999999999999,-,28,41.299999999999997,-,12,05,20110507,NaN,NaN
951844,rno=12&jcd=05&hd=20110507,6,112.599999999999994,NaN,東日本大震災被災地支援競走第４６回東京スポーツ賞,0,5,4089,0,4,1,30,53.5,0,0,0.18,6.15,43.899999999999999,0.000000000000000,5.90,41.299999999999997,0.000000000000000,25,38.200000000000003,-,60,26.899999999999999,-,12,05,20110507,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996079,rno=9&jcd=13&hd=20200715,1,110.900000000000006,1.3,尼崎モーターボート大賞〜まくってちょ〜うだい！！〜,10,2,4477,0,205,3,32,51.3,0,0,0.15,7.40,52.710000000000001,79.069999999999993,7.60,55.320000000000000,72.340000000000003,14,55.100000000000001,67.35,32,36.000000000000000,56.00,9,13,20200715,1.2,1.4
996080,rno=9&jcd=13&hd=20200715,2,111.599999999999994,7.6,尼崎モーターボート大賞〜まくってちょ〜うだい！！〜,10,3,4573,0,425,6,31,51.2,1,0,0.15,7.14,55.100000000000001,71.430000000000007,7.70,65.000000000000000,70.000000000000000,42,48.780000000000001,65.85,55,34.880000000000003,55.81,9,13,20200715,1.5,2.0
996081,rno=9&jcd=13&hd=20200715,5,114.700000000000003,8.5,尼崎モーターボート大賞〜まくってちょ〜うだい！！〜,10,4,4375,1,1130,0,36,51.0,0,0,0.18,5.60,36.789999999999999,59.430000000000000,6.60,55.060000000000002,67.420000000000002,32,37.500000000000000,47.50,69,48.939999999999998,59.57,9,13,20200715,2.4,3.5
996082,rno=9&jcd=13&hd=20200715,3,113.299999999999997,15.2,尼崎モーターボート大賞〜まくってちょ〜うだい！！〜,10,5,4381,1,233,0,35,51.0,0,0,0.17,5.65,42.049999999999997,59.090000000000003,5.73,39.799999999999997,60.200000000000003,45,33.329999999999998,51.67,62,32.649999999999999,46.94,9,13,20200715,3.2,4.6


In [11]:
# sanitize rank
def int_parse(x):
    try:
        return int(x)
    except:
        return None

df["rank"] = df["rank"].apply(int_parse)
df = df[pd.notnull(df["rank"])]

In [12]:
# sanitize tansho_odds, wight, mean_st, mortor_win_in_3_prob, boat_win_in_3_prob

def sanitize(x):
    try:
        return float(x)
    except:
        try:
            return int(x)
        except:
            return None

for col in ["tansho_odds", "weight", "mean_st", "mortor_win_in_3_prob", "boat_win_in_3_prob", "rno", "jcd", "hd"]:
    df[col] = df[col].apply(sanitize)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1313895 entries, 951840 to 996083
Data columns (total 33 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   uniq_key               1313895 non-null  object 
 1   rank                   1313895 non-null  float64
 2   race_time              1009659 non-null  float64
 3   tansho_odds            800231 non-null   float64
 4   cup_name               1313895 non-null  object 
 5   kaijo_name             1313895 non-null  int64  
 6   waku_name              1313895 non-null  int64  
 7   touroku_no             1313895 non-null  int64  
 8   class_name             1313895 non-null  int64  
 9   racer_name             1313895 non-null  int64  
 10  country                1313895 non-null  int64  
 11  age                    1313895 non-null  int64  
 12  weight                 1313895 non-null  float64
 13  fnum                   1313895 non-null  int64  
 14  lnum          

In [13]:
df.shape

(1313895, 33)

In [216]:
df = df[pd.notnull(df["tansho_odds"])]

In [217]:
a = df.copy()
a["tansho_odds+waku_name"] = a["tansho_odds"] + a["waku_name"]
a["tansho_odds*waku_name"] = a["tansho_odds"] * a["waku_name"]
 
# df["zenkoku_win_in_1_prob/waku_name"] = df["zenkoku_win_in_1_prob"] / df["waku_name"]

bs = []
for racer_name in a["racer_name"].unique():
    b = a[a["racer_name"] == racer_name][["hd", "racer_name", "race_time", "rank", "waku_name"]]
    b["p1_race_time"] = b["race_time"].shift(+1)
    b["p2_race_time"] = b["race_time"].shift(+2)
    b["p3_race_time"] = b["race_time"].shift(+3)
    b["prev1_rank"] = b["rank"].shift(+1)
    b["prev2_rank"] = b["rank"].shift(+2)
    b["prev3_rank"] = b["rank"].shift(+3)
    
    b["prev_min_rank"] = b[["prev1_rank", "prev2_rank", "prev3_rank"]].min(axis=1)
    b["prev_mean_rank"] = b[["prev1_rank", "prev2_rank", "prev3_rank"]].mean(axis=1)
    b["prev_median_rank"] = b[["prev1_rank", "prev2_rank", "prev3_rank"]].median(axis=1)
    bs.append(b.drop(["rank","race_time", "waku_name"], axis=1))
bs = pd.concat(bs)
a = pd.merge(a, bs, on=["hd", "racer_name"], how="left")


In [238]:
holddays = a["hd"]
size=len(holddays)
trn_holddays = set(holddays[:int(0.8*size)])
import random
random.seed(13)
# trn_cup = set(random.sample(cup_names.tolist(), int(0.8*size)))
val_holddays = {c for c in holddays if c not in trn_holddays}

trn_y = a[a.hd.apply(lambda x:x in trn_holddays)]["rank"].apply(lambda x:x<=3)
val_y = a[a.hd.apply(lambda x:x in val_holddays)]["rank"].apply(lambda x:x<=3)

val_y

1062130     True
1062131     True
1062132    False
1062133    False
1062134     True
           ...  
1326328    False
1326329     True
1326330     True
1326331    False
1326332    False
Name: rank, Length: 264203, dtype: bool

In [239]:
trn_x = a[a.hd.apply(lambda x:x in trn_holddays)].drop(["uniq_key", "race_time", "rank", "cup_name"], axis=1)
val_x = a[a.hd.apply(lambda x:x in val_holddays)].drop(["uniq_key", "race_time", "rank", "cup_name"], axis=1)

trn_x

,tansho_odds,kaijo_name,waku_name,touroku_no,class_name,racer_name,country,age,weight,fnum,lnum,mean_st,zenkoku_win_in_1_prob,zenkoku_win_in_2_prob,zenkoku_win_in_3_prob,touchi_win_in_1_prob,touchi_win_in_2_prob,touchi_win_in_3_prob,mortor_win_in_1_prob,mortor_win_in_2_prob,mortor_win_in_3_prob,boat_win_in_1_prob,boat_win_in_2_prob,boat_win_in_3_prob,rno,jcd,hd,fukusho_min,fukusho_max,tansho_odds+waku_name,tansho_odds*waku_name,p1_race_time,p2_race_time,p3_race_time,prev1_rank,prev2_rank,prev3_rank,prev_min_rank,prev_mean_rank,prev_median_rank
0,1.000000000000000,6,1,3589,0,1150,6,43,52.399999999999999,0,0,0.16,6.29,46.149999999999999,63.079999999999998,4.25,37.500000000000000,37.500000000000000,21,27.370000000000001,43.159999999999997,36,31.960000000000001,49.770000000000003,1.0,23.0,20170308.0,1.0,1.0,2.000000000000000,1.000000000000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.000000000000000,6,1,3589,0,1150,6,43,52.399999999999999,0,0,0.16,6.29,46.149999999999999,63.079999999999998,4.25,37.500000000000000,37.500000000000000,21,27.370000000000001,43.159999999999997,36,31.960000000000001,49.770000000000003,1.0,23.0,20170308.0,1.0,1.0,2.000000000000000,1.000000000000000,114.700000000000003,NaN,NaN,1.0,NaN,NaN,1.0,1.000000000000000,1.0
2,9.500000000000000,6,2,3847,2,1476,2,40,56.100000000000001,0,0,0.15,3.67,13.949999999999999,31.399999999999999,1.89,0.000000000000000,0.000000000000000,13,33.079999999999998,50.000000000000000,52,39.450000000000003,55.500000000000000,1.0,23.0,20170308.0,1.3,2.4,11.500000000000000,19.000000000000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7.600000000000000,6,3,3411,2,581,7,47,52.299999999999997,0,0,0.16,4.35,23.609999999999999,33.329999999999998,5.58,36.840000000000003,52.630000000000003,53,19.660000000000000,32.479999999999997,35,39.090000000000003,57.270000000000003,1.0,23.0,20170308.0,1.3,2.3,10.600000000000000,22.799999999999997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7.600000000000000,6,3,3411,2,581,7,47,52.299999999999997,0,0,0.16,4.35,23.609999999999999,33.329999999999998,5.58,36.840000000000003,52.630000000000003,53,19.660000000000000,32.479999999999997,35,39.090000000000003,57.270000000000003,1.0,23.0,20170308.0,1.3,2.3,10.600000000000000,22.799999999999997,119.500000000000000,NaN,NaN,3.0,NaN,NaN,3.0,3.000000000000000,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1062125,2.800000000000000,9,4,4547,0,724,6,31,49.000000000000000,0,0,0.14,5.92,30.379999999999999,48.100000000000001,7.25,58.950000000000003,69.469999999999999,26,29.410000000000000,53.920000000000002,58,19.230000000000000,33.329999999999998,9.0,2.0,20200104.0,1.5,3.5,6.800000000000000,11.199999999999999,109.000000000000000,110.900000000000006,111.700000000000003,1.0,4.0,5.0,1.0,3.333333333333333,4.0
1062126,2.800000000000000,9,4,4547,0,724,6,31,49.000000000000000,0,0,0.14,5.92,30.379999999999999,48.100000000000001,7.25,58.950000000000003,69.469999999999999,26,29.410000000000000,53.920000000000002,58,19.230000000000000,33.329999999999998,9.0,2.0,20200104.0,1.5,3.5,6.800000000000000,11.199999999999999,NaN,109.000000000000000,110.900000000000006,5.0,1.0,4.0,1.0,3.333333333333333,4.0
1062127,18.800000000000001,9,5,3353,1,559,6,51,51.000000000000000,0,0,0.17,5.76,40.630000000000003,63.539999999999999,5.69,37.180000000000000,58.969999999999999,41,35.869999999999997,47.829999999999998,30,38.890000000000001,53.329999999999998,9.0,2.0,20200104.0,2.7,6.5,23.800000000000001,94.000000000000000,109.799999999999997,NaN,110.000000000000000,2.0,6.0,2.0,2.0,3.333333333333333,2.0
1062128,18.800000000000001,9,5,3353,1,559,6,51,51.000000000000000,0,0,0.17,5.76,40.630000000000003,63.539999999999999,5.69,37.180000000000000,58.969999999999999,41,35.869999999999997,47.829999999999998,30,38.890000000000001,53.329999999999998,9.0,2.0,20200104.0,2.7,6.5,23.800000000000001,94.000000000000000,NaN,109.799999999999997,NaN,5.0,2.0,6.0,2.0,4.333

In [240]:
trn_y

0           True
1           True
2           True
3           True
4           True
           ...  
1062125     True
1062126     True
1062127    False
1062128    False
1062129    False
Name: rank, Length: 1062130, dtype: bool

In [241]:
param = {
            "objective": "binary",
            "metric": "auc",
            "max_depth": 20,
            "num_leaves": 20,
            "learning_rate": 0.1,
            #"bagging_fraction": 0.99,
            #"feature_fraction": 0.99,
            #"lambda_l1": 0.3,
            #"lambda_l2": 0.3,
            "bagging_seed": 777,
            "verbosity": -1,
            "seed": 777,
            # 'max_bin': 512
}

In [242]:
def train(trn_x, trn_y, val_x, val_y, category, param, verbose, early_stopping_rounds, n_estimators):

    print(f'now start to train')
    trn_data = lgb.Dataset(trn_x, label=trn_y, categorical_feature=category)
    val_data = lgb.Dataset(val_x, label=val_y, categorical_feature=category)
    num_round = n_estimators
    clf = lgb.train(param, trn_data, num_round, valid_sets=[
        trn_data, val_data], verbose_eval=verbose, early_stopping_rounds=early_stopping_rounds)
    #oof_predictions[xs_val_idx] = clf.predict(xs_val)
    eval_loss = roc_auc_score(val_y, clf.predict(val_x))
    print(f'end eval_loss={eval_loss}')
    return clf

In [243]:
category = [idx for idx, c in enumerate(trn_x.columns) if c in {"class_name", "kaijo_name", "racer_name", "country", "fnum", "lnum"}]
clf = train(trn_x, trn_y, val_x, val_y, category, param, 1, 50, 10000)

now start to train
[1]	training's auc: 0.77554	valid_1's auc: 0.781759
Training until validation scores don't improve for 50 rounds
[2]	training's auc: 0.784008	valid_1's auc: 0.789951
[3]	training's auc: 0.785517	valid_1's auc: 0.791657
[4]	training's auc: 0.786845	valid_1's auc: 0.79298
[5]	training's auc: 0.788467	valid_1's auc: 0.794484
[6]	training's auc: 0.789309	valid_1's auc: 0.795323
[7]	training's auc: 0.790477	valid_1's auc: 0.796523
[8]	training's auc: 0.790815	valid_1's auc: 0.79689
[9]	training's auc: 0.791299	valid_1's auc: 0.797441
[10]	training's auc: 0.792003	valid_1's auc: 0.798105
[11]	training's auc: 0.792533	valid_1's auc: 0.798619
[12]	training's auc: 0.793444	valid_1's auc: 0.799481
[13]	training's auc: 0.793903	valid_1's auc: 0.799868
[14]	training's auc: 0.794351	valid_1's auc: 0.800338
[15]	training's auc: 0.794706	valid_1's auc: 0.800644
[16]	training's auc: 0.795514	valid_1's auc: 0.801379
[17]	training's auc: 0.79595	valid_1's auc: 0.801536
[18]	training's

In [244]:
imp = [{"column":c, "gain":g} for c,g in zip(trn_x.columns, clf.feature_importance(importance_type="gain"))]

pd.DataFrame(imp).sort_values(by=["gain"], ascending=False)

,column,gain
27,fukusho_min,504272.659294128417969
34,prev1_rank,330451.948448181152344
29,tansho_odds+waku_name,244247.701347351074219
30,tansho_odds*waku_name,201926.905681610107422
5,racer_name,149834.202022552490234
0,tansho_odds,138697.735496520996094
24,rno,101110.083957672119141
12,zenkoku_win_in_1_prob,89752.651042938232422
2,waku_name,45920.269271850585938
38,prev_mean_rank,8542.212982177734375
